In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from tqdm import tqdm

In [2]:
frequency = 'monthly'
column = 'ridership'

In [3]:
nyc_master_df = pd.read_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{frequency}.csv')

In [4]:
nyc_master_df.head()

,Unnamed: 0,year,month,station_complex_id,station_complex,borough,latitude,longitude,ridership,unique_time,...,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,0,2020,10,1,"Astoria-Ditmars Blvd (N,W)",Queens,40.775036,-73.912030,159932.0,2020-10-01,...,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.912034 40.775036)
1,1,2020,10,10,"49 St (N,R,W)",Manhattan,40.759900,-73.984140,114934.0,2020-10-01,...,Subway,40.759901,-73.984139,Uptown,Downtown,2.0,1.0,0.0,Uptown only,POINT (-73.984139 40.759901)
2,2,2020,10,100,"Hewes St (M,J)",Brooklyn,40.706870,-73.953430,34240.0,2020-10-01,...,Elevated,40.706870,-73.953431,Outbound,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.953431 40.70687)
3,3,2020,10,101,"Marcy Av (M,J,Z)",Brooklyn,40.708360,-73.957756,134937.0,2020-10-01,...,Elevated,40.708359,-73.957757,Outbound,Manhattan,1.0,1.0,1.0,NaN,POINT (-73.957757 40.708359)
4,4,2020,10,103,"Bowery (J,Z)",Manhattan,40.720280,-73.993910,37123.0,2020-10-01,...,Subway,40.720280,-73.993915,Brooklyn,Downtown,0.0,0.0,0.0,NaN,POINT (-73.993915 40.72028)


In [5]:
station_network_list = nyc_master_df['GTFS Stop ID'].unique()
station_network_list.sort()
time_interval = nyc_master_df['unique_time'].unique()
time_interval.sort()

In [6]:
def compute_time_series(col: str):

    if col not in nyc_master_df:
        raise ValueError('Column does not exist')
    temp_dict = {}

    if is_numeric_dtype(nyc_master_df[col]):
        print('is numeric')
    else:
        print('is not numeric')

    for station in tqdm(station_network_list):
        for time in time_interval:
            temp_df = nyc_master_df[(nyc_master_df['GTFS Stop ID'] == station) & (nyc_master_df['unique_time'] == time)]
            if is_numeric_dtype(temp_df[col]):
                val = temp_df[col].mean() # Takes mean of all rows
            else:
                val = temp_df[col].max()
            if station not in temp_dict:
                temp_dict[station] = {}
            temp_dict[station][time] = val
    return temp_dict

In [7]:
ts_matrix = pd.DataFrame(compute_time_series(column))

is not numeric


100%|██████████| 426/426 [00:43<00:00,  9.90it/s]


In [8]:
ts_matrix

,101,103,104,106,107,108,109,110,111,113,...,R41,R42,R43,R44,R45,S01,S03,S04,S30,S31
2020-07-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2020-08-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2020-09-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2020-10-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2020-11-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2020-12-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2021-01-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2021-02-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2021-03-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island
2021-04-01,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,Broadway - 7Av,...,4th Av,4th Av,4th Av,4th Av,4th Av,Franklin Shuttle,Franklin Shuttle,Franklin Shuttle,Staten Island,Staten Island


In [9]:
# Shortcut solution to set all NA values to 0
ts_matrix.fillna(0, inplace=True)

# TODO Imputation for NA values

In [10]:
ts_matrix.to_csv(f'Data/Master Sets/NYC Master/NYC_Time_Series_{frequency}_{column}.csv')